# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

### Imports

In [ ]:
# Imports
import os
import warnings
import tools
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional
from matplotlib import pyplot
# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Data

In [3]:
# Root CSV files directory
dirname = "./data/absolute/2D/"  

# Load data and print summary, if desired
x_train, x_val, x_test, y_train, y_val, y_test, labels = tools.load_from(dirname, verbose=False) 

In [ ]:
import winsound
def finished(num):
    frequency = 2000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    for i in range(0, num):
        winsound.Beep(frequency, duration)

## Training Stage
Configure the model and train it.

Metrics:
<div float="right">
    <img src="assets/accuracy.png" width="400"> 
    <img src="assets/precision_recall_formula.png" width="400">
</div>
<img src="assets/precision_recall.png" width="1000">


### <span style="color:blue"> Hyperparametertuned LSTM </span>
##### Here it is necessary to install the Keras-Tuner Module by executing:
#####  <span style="color:green"> via Conda:</span>
conda install -c conda-forge keras-tuner
#####  <span style="color:green"> for pip:</span>
pip install keras-tuner

Right now there are three different builds we are testing:
- classic LSTM
- CuDNNLSTM
- bidriectional LSTM


In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
from time import time, strftime


starttime= strftime("%Y_%m_%d_%H%M%S")
LOG_DIR = "C:\ML\Optimization_"f"{starttime}" #<-In Windows below Log_dir Path will maybe be too long for Windows to handle, so use a shorter path like this here
#LOG_DIR = "./Optimization_"f"{starttime}" # LOG_DIR holds json files with information and a model of each single trial

def build_model_lstm(hp):
    model = Sequential()
    
    model.add(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model



def build_model_CuDNNLSTM(hp):
    model = Sequential()
    

    
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model



def build_model_bdlstm(hp):
    model = Sequential()
    model.add(Bidirectional(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64),
                                        return_sequences=True),
                                        input_shape=(100, 86)))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True)))
    
    model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32))))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  optimizer=hp.Choice('optimizer',values=['Adamax']),
                  metrics=['accuracy']) 
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model






###   <span style="color:red">Necesarry only in case of using Nvidia GPU  </span>

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Different Keras-Tuner Approaches
### 1 - RandomSearch
Parameter of variables are ranomly used (number of layers, number of nodes) and "best" model is chosen.

In [ ]:
tuner  = RandomSearch(
    build_model_bdlstm,     #Function to use search in... See different builds above
    objective = "val_accuracy",  #Chooses "best model" looking for highest value of val_accuracy
    max_trials = 30,       # Number of different combinations tried Nodes and layers
    executions_per_trial = 1, 
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train,      #syntax just like in fit
                y= y_train,
            epochs=200,
            batch_size=32,
            validation_data=(x_val,y_val),
            verbose=2
            )

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

### 2 - Hyperband
Variation of RandomSearch http://jmlr.org/papers/volume18/16-558/16-558.pdf

In [ ]:
tuner  = Hyperband(
    build_model,
    objective = "val_accuracy",
    hyperband_iterations=2,
    max_epochs=150,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train, 
            y= y_train,
            batch_size=32,
            validation_data=(x_val,y_val))

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

In [ ]:
#Laut Randomsearch bestes Model am 23.06.2020

model = Sequential()
model.add(layers.LSTM(128, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True)) 
model.add(layers.LSTM(96))  
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.summary()

history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False,verbose=2)

In [ ]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(256, return_sequences=True),
                                        input_shape=(100, 86)))
    
model.add(layers.Bidirectional(layers.LSTM(192,return_sequences=True)))
    
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  optimizer='Adamax',
                  metrics='accuracy') 
model.summary()
#history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False,verbose=2)

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

## Export tuner object into pickle file
so it can be used in other scripts

In [ ]:
import pickle

with open(f"tuner_"f"{starttime}.pkl", "wb") as f:
    pickle.dump(tuner, f)
    

## Get best Trial from Tuner Object

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
bestmodel= tuner.hypermodel.build(best_hp)

bestmodel.summary()


In [ ]:
#tmp_chekpoints= "tmp\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"
tmp_chekpoints= "C:\\ML\\checkpoints\\tmp\\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"

#csv_log = tf.keras.callbacks.CSVLogger("log.csv", separator=',', append=False)
csv_log = tf.keras.callbacks.CSVLogger("C:\ML\logs\log.csv", separator=',', append=False)

#tb = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
tb = tf.keras.callbacks.TensorBoard(log_dir='C:\ML\logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=0, mode='max', baseline=None, restore_best_weights=True)
chk= tf.keras.callbacks.ModelCheckpoint(tmp_chekpoints, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='max', save_freq='epoch')


history = model.fit(x_train,y_train,epochs=200,batch_size=32, validation_data=(x_val,y_val),shuffle=False, verbose=2, callbacks=[csv_log, chk])

### Diagnostic Plots

The training history of your LSTM models can be used to diagnose the behavior of your model.

You can plot the performance of your model using the Matplotlib library. For example, you can plot training loss vs test loss as follows:

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.savefig("C:/ML/loss"f"{starttime}.png")
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='lower right')
pyplot.savefig("C:/ML/accuracy_"f"{starttime}.png")
pyplot.show()

#### Underfit Example
Running this example produces a plot of train and validation loss showing the characteristic of an underfit model. In this case, performance may be improved by increasing the number of training epochs.


<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png" width="400">


Running this example shows the characteristic of an underfit model that appears under-provisioned.
In this case, performance may be improved by increasing the capacity of the model, such as the number of memory cells in a hidden layer or number of hidden layers.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png" width="400">

#### Good Fit Example
Running the example creates a line plot showing the train and validation loss meeting.
Ideally, we would like to see model performance like this if possible, although this may not be possible on challenging problems with a lot of data.

<img src="assets/Diagnostic-Line-Plot-Showing-a-Good-Fit-for-a-Model.png" width="400">

#### Overfit Example
Running this example creates a plot showing the characteristic inflection point in validation loss of an overfit model.
This may be a sign of too many training epochs.
In this case, the model training could be stopped at the inflection point. Alternately, the number of training examples could be increased.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png" width="400">

### Evaluate

In [ ]:
#model = tf.keras.models.load_model('./tmp/epoch49-0.90-0.39.hdf5')


#bestmodel.evaluate(x=x_test, y=y_test, verbose=2)
model.evaluate(x=x_test, y=y_test, verbose=2)


### Save model

In [ ]:
bestmodel.save("sign_lang_recognition_tuned.h5")